In [4]:
# Imports
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
from langchain_community.tools import WikipediaQueryRun, ArxivQueryRun
from langchain_classic.chains import RetrievalQA
import os
from dotenv import load_dotenv

In [6]:
load_dotenv()

#Provide your key directly
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

In [49]:
# Step 2: Define the state schema
class State(TypedDict):
    query: str
    retriever: object
    route: str
    source: str
    final_answer: str

In [32]:
# Step 3: Create graph
graph = StateGraph(State)

In [10]:
from langchain_classic.output_parsers import StructuredOutputParser, ResponseSchema

In [13]:

# 2️⃣ Output parser for routing
schemas = [
    ResponseSchema(name="route", description="RAG, WIKI, ARXIV, or LLM"),
    ResponseSchema(name="answer", description="Answer if LLM is chosen, else empty string")
]
parser = StructuredOutputParser.from_response_schemas(schemas)



In [57]:
# --- Node functions ---

def router_node(state):
    query = state["query"]
    prompt = f"""
Decide which source to use for this question:
- RAG: AWS/AI Practitioner PDF and AWS related questions
- WIKI: Wikipedia/general knowledge
- ARXIV: Research papers
- LLM: Anything else

Respond with exactly one word: RAG, WIKI, ARXIV, or LLM

Question: {query}
"""
    route = llm.invoke(prompt).content.strip().upper()
    state["route"] = route
    return state

def rag_node(state):
    retriever = state.get("retriever")
    if not retriever:
        state["answer"] = "No retriever available for RAG."
    else:
        state["answer"] = RetrievalQA.from_chain_type(llm=llm, retriever=retriever).run(state["query"])
    state["source"] = "RAG"
    return state

def wiki_node(state):
    tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    state["answer"] = tool.run(state["query"])
    state["source"] = "WIKI"
    return state

def arxiv_node(state):
    api = ArxivAPIWrapper()
    state["answer"] = api.run(state["query"])
    state["source"] = "ARXIV"
    return state

def llm_node(state):
    state["answer"] = llm.predict(state["query"])
    state["source"] = "LLM"
    return state

def refine_node(state):
    prompt = f"""
Refine and summarize the answer clearly.
Question: {state['query']}
Answer: {state['answer']}
"""
    state["answer"] = llm.invoke(prompt).content
    return state


In [51]:
# --- Graph setup ---

State = dict
graph = StateGraph(State)

# Add nodes
graph.add_node("router", router_node)
graph.add_node("rag", rag_node)
graph.add_node("wiki", wiki_node)
graph.add_node("arxiv", arxiv_node)
graph.add_node("llm", llm_node)
graph.add_node("refine", refine_node)


In [52]:
# Conditional edges from router
graph.add_conditional_edges(
    "router",
    lambda s: s["route"],
    {
        "RAG": "rag",
        "WIKI": "wiki",
        "ARXIV": "arxiv",
        "LLM": "llm"
    }
)



In [53]:
# All tool nodes go to refine
graph.add_edge("rag", "refine")
graph.add_edge("wiki", "refine")
graph.add_edge("arxiv", "refine")
graph.add_edge("llm", "refine")

# End the workflow
graph.add_edge("refine", END)

In [54]:
# Set entry point and compile
graph.set_entry_point("router")
workflow = graph.compile()


In [58]:
# --- Example usage ---
query = "What is AWS sagemaker?"
result = workflow.invoke({"query": query, "retriever": None})
print("\nSource:", result["source"])
print("Answer:\n", result["answer"])


Source: RAG
Answer:
 AWS SageMaker is a fully managed service that enables developers and data scientists to build, train, and deploy machine learning models quickly and efficiently. It provides a range of tools and features for data preparation, model training, tuning, and deployment, making it easier to integrate machine learning into applications.
